# notes
- UCI ML News Aggregator Dataset contains headlines
- categories for over 400k news articles.
- currently uses multinomial scikit learn

In [1]:
# get some libraries that will be useful
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)x

# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder


# grab the data
news = pd.read_csv("../data_news_aggregator/news_data_uci.csv")

In [5]:
#news.head() # let's take a look at our data

One thing we'll want to do is normalize the TITLE column a bit: remove punctuation and lowercase everything. This will give us a smaller set of words, which will decrease the size of our model, and ensure that words are treated the same even if they occur capitalized at the beginning of the headline or lowercase in the middle.

In [2]:
from train import normalize_text
news['TEXT'] = [normalize_text(s) for s in news['TITLE']]
#print(len(news['TEXT']),news['TEXT'])

# format into matrix

In [37]:
# pull the data into vectors
tits = news["TEXT"][:1000]
stories = news["STORY"][:1000]
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(tits)

encoder = LabelEncoder()
y = encoder.fit_transform(stories)

print("x_shape,y_shape",x.shape,y.shape)

x_shape,y_shape (1000, 1816) (1000,)


In [38]:
print(tits.shape)
print(tits[0])
print(tits[1])
analyze = vectorizer.build_analyzer()
voc = set(analyze(tits[0]))
for i in range(len(tits)):
    voc.update(analyze(tits[i]))
print(len(sorted(voc)))

(1000,)
fed official says weak data caused by weather should not slow taper
fed's charles plosser sees high bar for change in pace of tapering
1816


In [39]:
nb = MultinomialNB()
nb.fit(x, y)
nb.score(x, y)

0.97499999999999998

In [47]:
vectorizer = CountVectorizer(vocabulary=voc)
print(tits[0])
x_1 = vectorizer.fit_transform([tits[0]])
print("x_1",x_1.shape)
y_1 = nb.predict(x_1)
print("y_1", y_1, y[0],encoder.inverse_transform(y_1))

fed official says weak data caused by weather should not slow taper
x_1 (1, 1816)
y_1 [11] 11 ['ddUyU0VZz0BRneMioxUPQVP6sIxvM']
